# Importing Libraries

In [4]:
import pandas as pd
from datetime import datetime
from math import ceil

# Reading Dataset

In [5]:
dataset = pd.read_csv('hb_bid_data.tsv', sep='\t')

# A. Print the count of users, sessions, pageviews and auctions.

In [6]:
print('Users: ', len(pd.unique(dataset['userId'])))

print('Sessions: ', len(pd.unique(dataset['sessionId'])))

print('Page Views: ', len(pd.unique(dataset['pageviewId'])))

print('Auctions: ', len(pd.unique(dataset['auctionId'])))

Users:  562
Sessions:  700
Page Views:  2067
Auctions:  2067


# B. List the top 5 countries with the highest average bid after filtering out all the cases with “noBids”

In [7]:
with_bid_condition = (dataset['status'] != 'noBid') & (dataset['country'] != '??')
with_bid_data = dataset[with_bid_condition]
average_bids = with_bid_data.groupby('country', as_index=False)['bid'].mean('bid').sort_values(by=['bid'], ascending=False)
average_bids.head()

,country,bid
45,ZA,0.916000
25,KR,0.460833
15,FI,0.354255
13,DK,0.320000
24,JM,0.235556


# C. List the maximum bid value from each bidders by different device types

In [9]:
max_bid = with_bid_data.groupby(['bidder','device'], as_index=False)['bid'].max('bid')
max_bid

,bidder,device,bid
0,aol,desktop,5.70
1,aol,mobile,6.26
2,aol,tablet,0.00
3,appnexus,desktop,2.56
4,appnexus,mobile,7.40
5,appnexus,tablet,0.22
6,audienceNetwork,mobile,2.74
7,conversant,desktop,1.45
8,conversant,mobile,2.97
9,conversant,tablet,0.13


# D. List the distribution of the number of auctions for every 5 minutes

In [10]:
dataset['datetime'] = dataset['timestamp']
dataset['datetime'] = pd.to_datetime(dataset['datetime'], unit='ms')
time_index = dataset.set_index('datetime')
time_index = time_index.resample('5T')['auctionId'].nunique()
time_index = time_index.to_frame()
time_index.reset_index(inplace=True)
tmin = time_index['datetime'].min()
time_index['Interval (in minutes)'] = time_index['datetime'] - tmin

time_index['Interval (in minutes)'] = time_index['Interval (in minutes)'].apply(lambda x : (x.seconds//60)+5)
result = time_index.drop('datetime',axis=1)
result.rename(columns={'auctionId':'Auction Count'}, inplace=True)

result


,Auction Count,Interval (in minutes)
0,1,5
1,109,10
2,127,15
3,95,20
4,66,25
5,73,30
6,99,35
7,119,40
8,138,45
9,163,50


# E. List the distribution of the bid latency for every 100ms interval

In [11]:
max_latency = dataset['bidLatency'].max()
max_latency
bins_limit = ceil(max_latency/10)
bins_limit

bins = [i*10 for i in range(0, bins_limit+10, 10)]
groups = dataset.groupby(pd.cut(dataset['bidLatency'], bins, right=False, include_lowest=True))
bid_latency_distribution = groups['bidLatency'].count().to_frame()

bid_latency_distribution = bid_latency_distribution.rename(columns={'bidLatency': 'Bid Count'}).reset_index()
bid_latency_distribution.rename(columns={'bidLatency': 'Interval (in milliseconds)'}, inplace=True)
bid_latency_distribution

,Interval (in milliseconds),Bid Count
0,"[0, 100)",9878
1,"[100, 200)",8763
2,"[200, 300)",12797
3,"[300, 400)",10248
4,"[400, 500)",8531
...,...,...
90,"[9000, 9100)",0
91,"[9100, 9200)",0
92,"[9200, 9300)",0
93,"[9300, 9400)",0


# F. Rank the bidder performance for the United States. In order to do that rely on a function f(x,y) = -0.0001x + 4 + y where, x = bidLatency and y = bid value

**Methodology:**
1. Filter records with country == 'US'
2. Add performance column using the given formula
3. Calculate the average performance of each bidder by grouping rows on bidders and then taking the mean of performance
4. Rank the bidders based on average performance


In [12]:
us_condition = dataset['country'] == 'US'
us_bidders = dataset[us_condition]
us_bidders = us_bidders[['bidder','country', 'bid', 'bidLatency']]
us_bidders['performance'] = -0.0001 * us_bidders['bidLatency'] + 4 + us_bidders['bid']
us_bidders =  us_bidders.groupby('bidder')['performance'].mean()
us_bidders = us_bidders.to_frame()
us_bidders.reset_index(inplace=True)
us_bidders["rank"] = us_bidders["performance"].rank(method ='first', ascending=False)
us_bidders = us_bidders.sort_values(by="rank", axis=0)
us_bidders

,bidder,performance,rank
2,audienceNetwork,5.788400,1.0
11,rubicon,4.137921,2.0
13,triplelift,4.110315,3.0
5,ix,4.110232,4.0
12,sovrn,4.090816,5.0
8,pubmatic,4.067456,6.0
7,openx,4.024822,7.0
3,conversant,4.013298,8.0
1,appnexus,4.012589,9.0
0,aol,4.005045,10.0
